In [1]:
# Import all necessary libraries

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
from tensorflow.python.ops.numpy_ops import np_config

import numpy as np
import os
from PIL import Image
import cv2
from PIL import ImageFilter
import textwrap, os

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops.numpy_ops import np_config

np_config.enable_numpy_behavior()
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image)
    return image

def preprocess_image(image):

    image = tf.image.resize(image, (32, 32))
    image = tf.cast(image, tf.float32) / 255.0
    image = image[..., :3]  # Remove the fourth channel if present

    return image

def create_dataset(image_paths, batch_size, is_training=True):
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if is_training:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    print(dataset)

    return dataset

# Load image paths
image_paths = [path for path in tf.io.gfile.glob(r'C:\Users\diego\Documents\cifar10\plane\*.png')]

# Split the image paths into a training set and validation set
train_image_paths = image_paths[:int(len(image_paths) * 0.8)]
val_image_paths = image_paths[int(len(image_paths) * 0.8):]

# Create the training and validation datasets
batch_size = 128
train_dataset = create_dataset(train_image_paths, batch_size, is_training=True)
val_dataset = create_dataset(val_image_paths, batch_size, is_training=False)


<PrefetchDataset element_spec=TensorSpec(shape=(128, 32, 32, None), dtype=tf.float32, name=None)>
<PrefetchDataset element_spec=TensorSpec(shape=(128, 32, 32, None), dtype=tf.float32, name=None)>


In [5]:
# Declaration of the hyperparameters

# data
num_epochs = 500 
image_size = 32
# resolution of Kernel Inception Distance measurement, see related section
kid_image_size = 75
padding = 0.25

# adaptive discriminator augmentation
max_translation = 0.125
max_rotation = 0.125
max_zoom = 0.25
target_accuracy = 0.85
integration_steps = 1000

# architecture
noise_size = 128
depth = 4
width = 128
leaky_relu_slope = 0.2
dropout_rate = 0.4

# optimization
batch_size = 128
learning_rate = 2e-3
beta_1 = 0.5
ema = 0.99

In [6]:
class KID(keras.metrics.Metric):
    def __init__(self, name="kid", **kwargs):
        super().__init__(name=name, **kwargs)

        # KID is estimated per batch and is averaged across batches
        self.kid_tracker = keras.metrics.Mean()

        # a pretrained InceptionV3 is used without its classification layer
        # transform the pixel values to the 0-255 range, then use the same
        # preprocessing as during pretraining
        self.encoder = keras.Sequential(
            [
                layers.InputLayer(input_shape=(image_size, image_size, 3)),
                layers.Rescaling(255.0),
                layers.Resizing(height=kid_image_size, width=kid_image_size),
                layers.Lambda(keras.applications.inception_v3.preprocess_input),
                keras.applications.InceptionV3(
                    include_top=False,
                    input_shape=(kid_image_size, kid_image_size, 3),
                    weights="imagenet",
                ),
                layers.GlobalAveragePooling2D(),
            ],
            name="inception_encoder",
        )

    def polynomial_kernel(self, features_1, features_2):
        feature_dimensions = tf.cast(tf.shape(features_1)[1], dtype=tf.float32)
        return (features_1 @ tf.transpose(features_2) / feature_dimensions + 1.0) ** 3.0

    def update_state(self, real_images, generated_images, sample_weight=None):
        real_features = self.encoder(real_images, training=False)
        generated_features = self.encoder(generated_images, training=False)

        # compute polynomial kernels using the two sets of features
        kernel_real = self.polynomial_kernel(real_features, real_features)
        kernel_generated = self.polynomial_kernel(
            generated_features, generated_features
        )
        kernel_cross = self.polynomial_kernel(real_features, generated_features)

        # estimate the squared maximum mean discrepancy using the average kernel values
        batch_size = tf.shape(real_features)[0]
        batch_size_f = tf.cast(batch_size, dtype=tf.float32)
        mean_kernel_real = tf.reduce_sum(kernel_real * (1.0 - tf.eye(batch_size))) / (
            batch_size_f * (batch_size_f - 1.0)
        )
        mean_kernel_generated = tf.reduce_sum(
            kernel_generated * (1.0 - tf.eye(batch_size))
        ) / (batch_size_f * (batch_size_f - 1.0))
        mean_kernel_cross = tf.reduce_mean(kernel_cross)
        kid = mean_kernel_real + mean_kernel_generated - 2.0 * mean_kernel_cross

        # update the average KID estimate
        self.kid_tracker.update_state(kid)

    def result(self):
        return self.kid_tracker.result()

    def reset_state(self):
        self.kid_tracker.reset_state()
   

In [7]:
# "hard sigmoid", useful for binary accuracy calculation from logits
def step(values):
    # negative values -> 0.0, positive values -> 1.0
    return 0.5 * (1.0 + tf.sign(values))


# augments images with a probability that is dynamically updated during training
class AdaptiveAugmenter(keras.Model):
    def __init__(self):
        super().__init__()

        # stores the current probability of an image being augmented
        self.probability = tf.Variable(0.0)

        # the corresponding augmentation names from the paper are shown above each layer
        # the authors show, that the blitting and geometric augmentations
        # are the most helpful in the low-data regime
        self.augmenter = keras.Sequential(
            [
                layers.InputLayer(input_shape=(image_size, image_size, 3)),
                # blitting/x-flip:
                layers.RandomFlip("horizontal"),
                # blitting/integer translation:
                layers.RandomTranslation(
                    height_factor=max_translation,
                    width_factor=max_translation,
                    interpolation="nearest",
                ),
                # geometric/rotation:
                layers.RandomRotation(factor=max_rotation),
                # geometric/isotropic and anisotropic scaling:
                layers.RandomZoom(
                    height_factor=(-max_zoom, 0.0), width_factor=(-max_zoom, 0.0)
                ),
            ],
            name="adaptive_augmenter",
        )

    def call(self, images, training):
        if training:
            augmented_images = self.augmenter(images, training)

            # during training either the original or the augmented images are selected
            # based on self.probability
            augmentation_values = tf.random.uniform(
                shape=(batch_size, 1, 1, 1), minval=0.0, maxval=1.0
            )
            augmentation_bools = tf.math.less(augmentation_values, self.probability)

            images = tf.where(augmentation_bools, augmented_images, images)
        return images

    def update(self, real_logits):
        current_accuracy = tf.reduce_mean(step(real_logits))

        # the augmentation probability is updated based on the dicriminator's
        # accuracy on real images
        accuracy_error = current_accuracy - target_accuracy
        self.probability.assign(
            tf.clip_by_value(
                self.probability + accuracy_error / integration_steps, 0.0, 1.0
            )
        )        

# DCGAN generator
def get_generator():

    # receive noise_input
    noise_input = keras.Input(shape=(noise_size,))
    # dense neural network
    x = layers.Dense(4 * 4 * width, use_bias=False)(noise_input)
    x = layers.BatchNormalization(scale=False)(x)
    x = layers.ReLU()(x)
    # reshape to pass it through the convolutional networks for "upsampling" and generating the desired images
    x = layers.Reshape(target_shape=(4, 4, width))(x)
    # first same convolution to increase number of channels while keeping the size
    x = layers.Conv2DTranspose(
        width*2, kernel_size=4, strides=1, padding="same", use_bias=False,
    )(x)
    x = layers.BatchNormalization(scale=False)(x)
    x = layers.ReLU()(x)    
    # convolutions to increase the size to generate the novel images
    x = layers.Conv2DTranspose(
        width*2, kernel_size=4, strides=2, padding="same", use_bias=False,
    )(x)
    x = layers.BatchNormalization(scale=False)(x)
    x = layers.ReLU()(x)     
    x = layers.Conv2DTranspose(
        width*2, kernel_size=4, strides=2, padding="same", use_bias=False,
    )(x)
    x = layers.BatchNormalization(scale=False)(x)
    x = layers.ReLU()(x) 
    # second same convolution to decrease number of channels while keeping the size
    x = layers.Conv2DTranspose(
        width, kernel_size=4, strides=1, padding="same", use_bias=False,
    )(x)
    x = layers.BatchNormalization(scale=False)(x)
    x = layers.ReLU()(x) 
    # last convolution with sigmoid activation to get the image 
    image_output = layers.Conv2DTranspose(
        3, kernel_size=4, strides=2, padding="same", activation="sigmoid",
    )(x)

    return keras.Model(noise_input, image_output, name="generator")


# DCGAN discriminator
def get_discriminator():
    image_input = keras.Input(shape=(image_size, image_size, 3))
    x = image_input
    for _ in range(depth):
        x = layers.Conv2D(
            width, kernel_size=4, strides=2, padding="same", use_bias=False,
        )(x)
        x = layers.BatchNormalization(scale=False)(x)
        x = layers.LeakyReLU(alpha=leaky_relu_slope)(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(dropout_rate)(x)
    output_score = layers.Dense(1)(x)

    return keras.Model(image_input, output_score, name="discriminator")


class GAN_ADA(keras.Model):
    def __init__(self):
        super().__init__()

        self.augmenter = AdaptiveAugmenter()
        self.generator = get_generator()
        self.ema_generator = keras.models.clone_model(self.generator)
        self.discriminator = get_discriminator()

        self.generator.summary()
        self.discriminator.summary()

    def compile(self, generator_optimizer, discriminator_optimizer, **kwargs):
        super().compile(**kwargs)

        # separate optimizers for the two networks
        self.generator_optimizer = generator_optimizer
        self.discriminator_optimizer = discriminator_optimizer

        self.generator_loss_tracker = keras.metrics.Mean(name="g_loss")
        self.discriminator_loss_tracker = keras.metrics.Mean(name="d_loss")
        self.real_accuracy = keras.metrics.BinaryAccuracy(name="real_acc")
        self.generated_accuracy = keras.metrics.BinaryAccuracy(name="gen_acc")
        self.augmentation_probability_tracker = keras.metrics.Mean(name="aug_p")
        self.kid = KID()

    @property
    def metrics(self):
        return [
            self.generator_loss_tracker,
            self.discriminator_loss_tracker,
            self.real_accuracy,
            self.generated_accuracy,
            self.augmentation_probability_tracker,
            self.kid,
        ]

    def generate(self, batch_size, training):
        latent_samples = tf.random.normal(shape=(batch_size, noise_size))
        # use ema_generator during inference
        if training:
            generated_images = self.generator(latent_samples, training)
        else:
            generated_images = self.ema_generator(latent_samples, training)
        return generated_images

    def adversarial_loss(self, real_logits, generated_logits):
        # this is usually called the non-saturating GAN loss

        real_labels = tf.ones(shape=(batch_size, 1))
        generated_labels = tf.zeros(shape=(batch_size, 1))

        # the generator tries to produce images that the discriminator considers as real
        generator_loss = keras.losses.binary_crossentropy(
            real_labels, generated_logits, from_logits=False
        )
        # the discriminator tries to determine if images are real or generated
        discriminator_loss = keras.losses.binary_crossentropy(
            tf.concat([real_labels, generated_labels], axis=0),
            tf.concat([real_logits, generated_logits], axis=0),
            from_logits=False,
        )

        return tf.reduce_mean(generator_loss), tf.reduce_mean(discriminator_loss)

    def train_step(self, real_images):
        real_images = self.augmenter(real_images, training=True)

        # use persistent gradient tape because gradients will be calculated twice
        with tf.GradientTape(persistent=True) as tape:
            generated_images = self.generate(batch_size, training=True)
            # gradient is calculated through the image augmentation
            generated_images = self.augmenter(generated_images, training=True)

            # separate forward passes for the real and generated images, meaning
            # that batch normalization is applied separately
            real_logits = self.discriminator(real_images, training=True)
            generated_logits = self.discriminator(generated_images, training=True)

            generator_loss, discriminator_loss = self.adversarial_loss(
                real_logits, generated_logits
            )

        # calculate gradients and update weights
        generator_gradients = tape.gradient(
            generator_loss, self.generator.trainable_weights
        )
        discriminator_gradients = tape.gradient(
            discriminator_loss, self.discriminator.trainable_weights
        )
        self.generator_optimizer.apply_gradients(
            zip(generator_gradients, self.generator.trainable_weights)
        )
        self.discriminator_optimizer.apply_gradients(
            zip(discriminator_gradients, self.discriminator.trainable_weights)
        )

        # update the augmentation probability based on the discriminator's performance
        self.augmenter.update(real_logits)

        self.generator_loss_tracker.update_state(generator_loss)
        self.discriminator_loss_tracker.update_state(discriminator_loss)
        self.real_accuracy.update_state(1.0, step(real_logits))
        self.generated_accuracy.update_state(0.0, step(generated_logits))
        self.augmentation_probability_tracker.update_state(self.augmenter.probability)

        # track the exponential moving average of the generator's weights to decrease
        # variance in the generation quality
        for weight, ema_weight in zip(
            self.generator.weights, self.ema_generator.weights
        ):
            ema_weight.assign(ema * ema_weight + (1 - ema) * weight)

        # KID is not measured during the training phase for computational efficiency
        return {m.name: m.result() for m in self.metrics[:-1]}

    def test_step(self, real_images):
        generated_images = self.generate(batch_size, training=False)

        self.kid.update_state(real_images, generated_images)

        # only KID is measured during the evaluation phase for computational efficiency
        return {self.kid.name: self.kid.result()}

    def plot_images(self, epoch=None, logs=None, num_rows=3, num_cols=6, interval=5):
        # plot random generated images for visual evaluation of generation quality
        if epoch is None or (epoch + 1) % interval == 0:
            num_images = num_rows * num_cols
            generated_images = self.generate(num_images, training=False)

            plt.figure(figsize=(num_cols * 2.0, num_rows * 2.0))
            for row in range(num_rows):
                for col in range(num_cols):
                    index = row * num_cols + col
                    plt.subplot(num_rows, num_cols, index + 1)
                    plt.imshow(generated_images[index])
                    plt.axis("off")
            plt.tight_layout()
            plt.show()
            plt.close()
            
            
            
# create and compile the model
model = GAN_ADA()
model.compile(
    generator_optimizer=keras.optimizers.Adam(learning_rate, beta_1),
    discriminator_optimizer=keras.optimizers.Adam(learning_rate, beta_1),
)

# save the best model based on the validation KID metric
checkpoint_path = "gan_model_ll"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor="val_kid",
    mode="min",
    save_best_only=True,
)


Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense (Dense)               (None, 2048)              262144    
                                                                 
 batch_normalization (BatchN  (None, 2048)             6144      
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 2048)              0         
                                                                 
 reshape (Reshape)           (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 4, 4, 256)        524288    
 nspose)                                                 

In [ ]:
# run training and plot generated images periodically
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=val_dataset,
    callbacks=[
        keras.callbacks.LambdaCallback(on_epoch_end=model.plot_images),
        checkpoint_callback,
    ],
)

In [ ]:
# save the trained model
tf.saved_model.save(model, "DCGAN_Cifar_Plane")


In [ ]:
def plot_images_best(model, num_rows=5, num_cols=4):
    # plot random generated images for visual evaluation of generation quality
    
    # Generate a large number of images, to choose the best from
    num_images = 5000
    generated_images = model.generate(num_images, training=False)

    # Extract the indices of the n best generated images based on the output from the discriminator
    scores = model.discriminator(generated_images)
    num_gen = num_rows * num_cols
    v, ind = tf.math.top_k(scores[:, 0], k=num_gen, sorted=False)
    print(v)
    print(ind)
    
    # Iterate over the best images and plot them all
    plt.figure(figsize=(num_cols * 2.0, num_rows * 2.0))
    for row in range(num_rows):
        for col in range(num_cols):
            index = row * num_cols + col
            ii = ind[index]
            plt.subplot(num_rows, num_cols, index + 1)
            plt.imshow(generated_images[ii])
            
            # Convert the tensor to a PIL Image
            pil_img = tf.keras.preprocessing.image.array_to_img(generated_images[ii])

            # Save the image to a file
            pil_img.save("generated_images_cifar_plane_"+str(index)+".png")
            
            plt.axis("off")
    plt.tight_layout()
    plt.show()
    plt.close()       
        
plot_images_best(model)        

In [ ]:
# Load ESRGAN model to increase the size of the generated images and improve their resolution 

os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"
# Declaring Constants
ESRGAN_path = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model_super = hub.load(ESRGAN_path)

In [ ]:
# Post Processing Functions to increase the size and resolution of the generated images

def postprocess(image_tensor):
    

    size = image_tensor.shape[1]
    new_size = size * 2
    # Upsample the image using TensorFlow
    resized_image = tf.image.resize(image_tensor, [new_size, new_size], method=tf.image.ResizeMethod.BICUBIC)

    # Apply denoising using OpenCV
    denoised_image = cv2.medianBlur(resized_image.numpy(), 3)

    #Applying sharpening filter
    pil_img = tf.keras.preprocessing.image.array_to_img(denoised_image)
    sharp = pil_img.filter(ImageFilter.SHARPEN)

    return sharp

def resize_res(image_tensor):

    # First denoise the image and prepare it for the ESRGAN model 
    denoised_image = cv2.medianBlur(image_tensor.numpy(), 3)
    image = tf.expand_dims(denoised_image, 0)
    image = image * 255

    # Improve the image with the ESRGAN model
    fake_image = model_super(image,training=False)

    # Prepare image for plotting and later use
    fake_image = tf.squeeze(fake_image)
    fake_image = fake_image / 255

    return fake_image


In [ ]:
def display_images(images):

    # Declare necessary variables for plotting configuration
    columns=5
    width=25
    height=10

    # Check if there are images for plotting
    if not images:
        print("No images to display.")
        return 

    # Plotting configuration    
    height = max(height, int(len(images)/columns) * height)
    plt.figure(figsize=(width, height))

    # Plot the images in subplots
    for i, image in enumerate(images):
        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        plt.imshow(image)

In [ ]:
# Function to plot the best images generated from the model and their resized versions

def plot_images_post(model, num_generated_images):
    # plot random generated images for visual evaluation of generation quality
    
    # Generate a large number of images, to choose the best from
    num_images = 5000
    generated_images = model.generate(num_images, training=False)

    # Extract the indices of the n best generated images based on the output from the discriminator
    scores = model.discriminator(generated_images)
    v, ind = tf.math.top_k(scores[:, 0], k=num_generated_images, sorted=True)
    print(v)
    print(ind)

    # Iterate over the best images, apply the resizing and improvement functions and plot them all
    for i in range(len(ind)):

        # Extract the best images based on the indices
        ii = ind[i]
        image_tensor = generated_images[ii]

        # First resizing and sharpening of the image (output size = 64*64*3)
        sharp = postprocess(image_tensor)

        # Second resizing and sharpening of the image (output size = 128*128*3)
        sharp_array = tf.keras.preprocessing.image.img_to_array(sharp)
        sharp1 = postprocess(sharp_array)

        # Improvement of the resolution and resizing using the ESRGAN model (output size = 128*128*3)
        res_image = resize_res(image_tensor)

        # Second improvement of the resolution and resizing using the ESRGAN model (output size = 256*256*3)
        resized_image_2 = tf.image.resize(image_tensor, [64, 64], method=tf.image.ResizeMethod.BICUBIC)
        res_image_256 = resize_res(resized_image_2)

        # Plotting of all images
        imgs = [image_tensor,sharp,sharp1,res_image,res_image_256]
        display_images(imgs)
    
num_generated_images = 20        
plot_images_post(model,num_generated_images)        